In [1]:
import random
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [2]:
#before starting to implment the Optimization code and
#before taking any further steps to make sure every thing goes on the right way we
# need to check if there is any missing values
data = pd.read_csv('UCI_Credit_Card.csv')
print(data.isna().sum())
print("duplicated values :","\n", data.duplicated().sum(), "\n")

ID                            0
LIMIT_BAL                     0
SEX                           0
EDUCATION                     0
MARRIAGE                      0
AGE                           0
PAY_0                         0
PAY_2                         0
PAY_3                         0
PAY_4                         0
PAY_5                         0
PAY_6                         0
BILL_AMT1                     0
BILL_AMT2                     0
BILL_AMT3                     0
BILL_AMT4                     0
BILL_AMT5                     0
BILL_AMT6                     0
PAY_AMT1                      0
PAY_AMT2                      0
PAY_AMT3                      0
PAY_AMT4                      0
PAY_AMT5                      0
PAY_AMT6                      0
default.payment.next.month    0
dtype: int64
duplicated values : 
 0 



In [3]:
# Initialization for binary representation 
def initialization(numFeatures, populationSize):
    return np.random.randint(2, size=(populationSize, numFeatures))

In [4]:
 #perform n-point crossover 
#def crossover(parent1, parent2, Pc=0.8, n_points=2):
  #  if random.random() < Pc:
        # Generate `n_points` unique random crossover points, sorted in ascending order.
   #     points = sorted(random.sample(range(1, len(parent1)), n_points))

        # Initialize children with copies of the parents.
    #    child1, child2 = parent1.copy(), parent2.copy()

        # Perform crossover by alternating segments between parents.
     #   for i in range(len(points)):
     #       if i % 2 == 0:  # Swap segments at even intervals
      #          start = points[i]
      #          end = points[i + 1] if i + 1 < len(points) else len(parent1)
      #          child1[start:end], child2[start:end] = parent2[start:end], parent1[start:end]
        
      #  return child1, child2
    #else:
        # No crossover; return copies of the parents.
    #    return parent1.copy(), parent2.copy()


In [4]:
# Uniform Crossover
def crossover(parent1, parent2, Pc):
    if random.random() < Pc:
        mask = np.random.randint(2, size=len(parent1))
        child1 = np.where(mask == 1, parent1, parent2)
        child2 = np.where(mask == 0, parent1, parent2)
        return child1, child2
    else:
        return parent1.copy(), parent2.copy()

In [5]:
# Bit-Flipping Mutation
def mutation(solution, Pm):
    for i in range(len(solution)):
        if random.random() < Pm:
            solution[i] = 1 - solution[i]  # Flip bit
    return solution

In [6]:
def evaluation(X, y, solution):
    # Ensure solution is a numpy array
    solution = np.array(solution)
    
    # Scale the features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Validate solution shape
    if solution.shape[0] != X_scaled.shape[1]:
        raise ValueError("Solution length must match the number of features in X_scaled.")
    
    # Select features based on the solution vector
    selected_features = X_scaled[:, solution == 1]
    
    # Handle the case of no features selected
    if selected_features.shape[1] == 0:
        return 0  # Return a penalty score for invalid feature selection
    
    # Split the data
    X_train, X_valid, y_train, y_valid = train_test_split(selected_features, y, test_size=0.2, random_state=42)
    
    # Train and evaluate the model
    clf = DecisionTreeClassifier(random_state=42)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_valid)
    
    # Return accuracy score
    return accuracy_score(y_valid, y_pred)

In [8]:
#def tournament_selection(population, fitness_scores, tournament_size=3):
   # selected = []
   # for _ in range(2):  # Select two parents
   #     candidates = np.random.choice(len(population), tournament_size, replace=False)
    #    best_candidate = max(candidates, key=lambda c: fitness_scores[c])
    #    selected.append(population[best_candidate])
    #return np.array(selected)


#survival selection
#def mu_plus_lambda_selection(parents, offspring, fitness_parents, fitness_offspring, mu):
 #   combined_population = np.vstack((parents, offspring))
 #   combined_fitness = np.hstack((fitness_parents, fitness_offspring))
  #  sorted_indices = np.argsort(combined_fitness)[::-1]  # Sort in descending order
  #  return combined_population[sorted_indices[:mu]], combined_fitness[sorted_indices[:mu]]

In [7]:
# Fitness-Proportionate Selection
def fitness_proportionate_selection(population, fitness_scores):
    total_fitness = sum(fitness_scores)
    probabilities = [fitness / total_fitness for fitness in fitness_scores]
    cumulative_probabilities = np.cumsum(probabilities)

    def select_parent():
        random_value = random.random()
        for index, cumulative_probability in enumerate(cumulative_probabilities):
            if random_value <= cumulative_probability:
                return population[index]

    parent1 = select_parent()
    parent2 = select_parent()
    while np.array_equal(parent1, parent2):
        parent2 = select_parent()

    return parent1, parent2


def round_robin_survivor_selection(population, fitness_scores, num_survivors):
    #pair individuals with their fitness score 
    paired_population = list(zip(population, fitness_scores))
    #sort by fitness scores in descending order(higher fitness is better) 
    paired_population.sort(key=lambda x: x[1], reverse=True)
    #Initialize survivors
    survivors = []
    index = 0
    #select survivors in round-robin manner
    while len(survivors) < num_survivors:
        survivors.append(paired_population[int(index % len(paired_population))])
        index += 1
    
    selected_population, selected_fitness_scores = zip(*survivors)
    return list(selected_population), list(selected_fitness_scores)

In [8]:
# Genetic Algorithm
def EA(X, y, population_size, num_iterations, Pc, Pm, mu):
    num_features = X.shape[1]
    population = initialization(num_features, population_size)
    fitness_scores = np.array([evaluation(X, y, sol) for sol in population])

    for generation in range(num_iterations):
        # Recompute fitness scores
        fitness_scores = np.array([evaluation(X, y, sol) for sol in population])
        sorted_indices = np.argsort(fitness_scores)[::-1]

        # Convert population to NumPy array for sorting
        population = np.array(population)[sorted_indices]
        fitness_scores = fitness_scores[sorted_indices]

        max_accuracy = fitness_scores[0]
        print(f"Generation {generation + 1}: Accuracy = {max_accuracy}")

        new_population = []
        while len(new_population) < population_size:
            parent1, parent2 = fitness_proportionate_selection(population, fitness_scores)
            child1, child2 = crossover(parent1, parent2, Pc)
            child1 = mutation(child1, Pm)
            child2 = mutation(child2, Pm)
            new_population.extend([child1, child2])

        # Evaluate offspring fitness
        offspring_fitness_scores = np.array([evaluation(X, y, sol) for sol in new_population])
        
        # Combine old and new populations
        combined_population = list(population) + new_population
        combined_fitness_scores = list(fitness_scores) + list(offspring_fitness_scores)

        # Perform selection
        population, fitness_scores = round_robin_survivor_selection(
            combined_population, combined_fitness_scores, mu
        )

        # Ensure fitness_scores are recomputed for the new population
        fitness_scores = np.array([evaluation(X, y, sol) for sol in population])

    best_individual = population[0]
    best_fitness = fitness_scores[0]
    return best_individual, best_fitness

In [9]:
# Load data and split features/target
data = pd.read_csv('UCI_Credit_Card.csv')
X = data.drop(columns=['default.payment.next.month'])  # Features
y = data['default.payment.next.month']  # Target

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.values)

# Set parameters for the Genetic Algorithm
populationSize = 10
numIterations = 100
Pc = 0.8  # Crossover probability
Pm = 0.05  # Mutation probability
mu = 10  # Number of top individuals to retain

# Perform Genetic Algorithm for feature selection
best_individual, best_fitness = EA(X, y, populationSize, numIterations, Pc, Pm, mu)

# Print the results
print("Best Individual (Selected Features):", best_individual)
print("Best Fitness (Accuracy):", best_fitness)

Generation 1: Accuracy = 0.7261666666666666
Generation 2: Accuracy = 0.7261666666666666
Generation 3: Accuracy = 0.7351666666666666
Generation 4: Accuracy = 0.7351666666666666
Generation 5: Accuracy = 0.7351666666666666
Generation 6: Accuracy = 0.7351666666666666
Generation 7: Accuracy = 0.7351666666666666
Generation 8: Accuracy = 0.7351666666666666
Generation 9: Accuracy = 0.7351666666666666
Generation 10: Accuracy = 0.7351666666666666
Generation 11: Accuracy = 0.7351666666666666
Generation 12: Accuracy = 0.7368333333333333
Generation 13: Accuracy = 0.7401666666666666
Generation 14: Accuracy = 0.7401666666666666
Generation 15: Accuracy = 0.7401666666666666
Generation 16: Accuracy = 0.7401666666666666
Generation 17: Accuracy = 0.7401666666666666
Generation 18: Accuracy = 0.7401666666666666
Generation 19: Accuracy = 0.7401666666666666
Generation 20: Accuracy = 0.7401666666666666
Generation 21: Accuracy = 0.7401666666666666
Generation 22: Accuracy = 0.7401666666666666
Generation 23: Accu